In [19]:
import pandas as pd
import glob

In [35]:
"Reload the CSV file after code execution state reset"
financial_files_path = "./data_csv/Financial_*.csv"
all_files = glob.glob(financial_files_path)

In [ ]:
df_list = []
for file in all_files:
    temp_df = pd.read_csv(file)
    df_list.append(temp_df)

In [25]:
merged_df = pd.concat(df_list, ignore_index= True)
print(merged_df[['CERT', 'NAMEFULL']].head())

   CERT                             NAMEFULL
0     9                  UNION TRUST COMPANY
1    14  STATE STREET BANK AND TRUST COMPANY
2    35                           AUBURNBANK
3    39            ROBERTSON BANKING COMPANY
4    41                   PHENIX-GIRARD BANK


In [33]:
len(merged_df.columns)

158

In [41]:
merged_df[['NAASSET']]

,NAASSET
0,1791
1,3975
2,21
3,1839
4,48
...,...
168640,2749
168641,2488
168642,1851
168643,0


In [30]:
total_assets_df = pd.read_csv("./data_csv/Total_assets_2006_Q1.csv")

In [27]:
# Recalculate CAMELS ratios using correct column names with suffix '_x'
merged_df["equity_to_assets"] = merged_df["EQV_x"] / merged_df["NAASSET_x"]
merged_df["tier1_capital_ratio"] = merged_df["IDT1CER_x"]
merged_df["loan_loss_reserve_to_loans"] = merged_df["LNATRESR_x"] / merged_df["NTLNLSR_x"]
merged_df["noncurrent_loans_to_loans"] = merged_df["NCLNLSR_x"] / merged_df["NTLNLSR_x"]
merged_df["net_loans_to_assets"] = merged_df["LNLSNET_x"] / merged_df["NAASSET_x"]
merged_df["efficiency_ratio"] = merged_df["EINTEXP_x"] / merged_df["INTINC_x"]
merged_df["noninterest_income_to_assets"] = merged_df["NONII_x"] / merged_df["NAASSET_x"]
merged_df["return_on_assets"] = merged_df["ROA_x"]
merged_df["return_on_equity"] = merged_df["ROE_x"]
merged_df["net_interest_margin"] = merged_df["NIM_x"]
merged_df["net_loans_to_deposits"] = merged_df["LNLSNET_x"] / merged_df["DEPDOM_x"]
merged_df["core_deposits_to_assets"] = merged_df["COREDEP_x"] / merged_df["NAASSET_x"]
merged_df["securities_to_assets"] = merged_df["IGLSEC_x"] / merged_df["NAASSET_x"]

KeyError: 'EQV_x'